In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils.text_processor import preprocess_text
from utils.utils import *
from pathlib import Path
import pandas as pd


In [140]:
china_df = pd.read_csv(str(Path.cwd().parent)+ '/data/China_df.csv')
hk_df = pd.read_csv(str(Path.cwd().parent) + '/data/HongKong_df.csv')
taiwan_df = pd.read_csv(str(Path.cwd().parent) + '/data/taiwan_df.csv')


In [141]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context


nltk.download('punkt_tab')
nltk.download('stopwords')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\limyi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\limyi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [142]:
  # Check if posts is indeed a DataFrame
def process_column(posts, include_selftext = False):
    if not isinstance(posts, pd.DataFrame):
      raise ValueError("Input `posts` should be a pandas DataFrame.")
    texts = posts['title'].apply(preprocess_text)
    if include_selftext:
      # Add 'selftext' to each text if requested
      texts += ' ' + posts['selftext'].apply(preprocess_text)

    # Convert texts to a list format for further analysis
    texts = texts.tolist()
    return texts

def top_n_terms(tfidf_matrix, feature_names, top_n=10):
    mean_tfidf = tfidf_matrix.mean(axis=0).tolist()[0]

    tfidf_scores = list(zip(feature_names, mean_tfidf))

    tfidf_scores = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)

    return tfidf_scores[:top_n]

In [143]:
china_text = process_column(china_df)
ch_tfidf_matrix, chi_feature_names = generate_tfidf_matrix(china_text)
china_terms = top_n_terms(ch_tfidf_matrix, chi_feature_names)
china_terms

[('get', 0.01333039119919301),
 ('help', 0.012256770012414573),
 ('travel', 0.011604797339666018),
 ('shanghai', 0.011460644455774563),
 ('day', 0.011336978649633268),
 ('beijing', 0.011160581274037469),
 ('use', 0.0105954248048162),
 ('make', 0.009351604322744053),
 ('like', 0.009161583847548565),
 ('people', 0.008849990498166002)]

In [144]:
taiwan_text = process_column(taiwan_df)
tw_tfidf_matrix, tw_feature_names = generate_tfidf_matrix(taiwan_text)
tw_terms = top_n_terms(tw_tfidf_matrix, tw_feature_names)
tw_terms

[('taipei', 0.04888235139069697),
 ('typhoon', 0.02169607534759792),
 ('day', 0.01917809839020844),
 ('travel', 0.015629144313575837),
 ('question', 0.015273628786330425),
 ('look', 0.01448716191344036),
 ('kaohsiung', 0.014308193133987903),
 ('best', 0.012743342652405714),
 ('buy', 0.012541095277490068),
 ('get', 0.011752293342682145)]

In [145]:
hk_text = process_column(hk_df)
hk_tfidf_matrix, hk_feature_names = generate_tfidf_matrix(hk_text)
hk_terms = top_n_terms(hk_tfidf_matrix, hk_feature_names)
hk_terms

[('buy', 0.015322746482871583),
 ('get', 0.013398167656073571),
 ('look', 0.013257383115877795),
 ('anyone', 0.01278438485449765),
 ('recommendation', 0.011940126666620227),
 ('question', 0.011699494604170638),
 ('best', 0.011450014123299912),
 ('store', 0.011275080533516378),
 ('travel', 0.01048618992652609),
 ('help', 0.010281285146268162)]